<a href="https://colab.research.google.com/github/Theieyrre/Hate-Speech-NLP/blob/main/Prominence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm import tqdm
from math import log

In [ ]:
import nltk
nltk.download('stopwords')
stop_word_list = nltk.corpus.stopwords.words('turkish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_word_list.append('rt')
stop_word_list.append('değil')
stop_word_list.append('bir')

In [ ]:
df_off = pd.read_csv('words.csv')
df_not = pd.read_csv('words-not.csv')

In [ ]:
df_off = df_off[~df_off.word.isin(stop_word_list)]
df_off.head()

,Unnamed: 0,word,count
6,6,06melihgokcek,152869
7,7,rterdogan,141650
8,8,olsun,131183
12,12,ayasofya,103141
15,15,tcsavunma,82686


In [ ]:
df_not = df_not[~df_not.word.isin(stop_word_list)]
df_not.head()

,Unnamed: 0,word,count
2,2,rterdogan,418320
4,4,cumhurbaşkanımız,279799
5,5,tcsavunma,241163
7,7,akparti,196692
9,9,erdoğan,168149


In [ ]:
len_off = df_off['count'].sum()
len_not = df_not['count'].sum()

In [ ]:
pr_dict = dict()
for index, row in tqdm(df_off.iterrows(), total=len(df_off), position=0, leave=True):
    t = row['word']
    tft_da = row['count']
    tft_da_a = tft_da / len_off
    word_in_not = df_not[df_not.word == t]
    tft_db = word_in_not.iloc[0].loc['count'] if len(word_in_not) else 0
    tft_db_b = tft_db / len_not
    pr = log(tft_da) * (2 * tft_da_a / (tft_da_a + tft_db_b) - 1)
    pr_dict[t] = pr

100%|██████████| 27209/27209 [01:35<00:00, 285.35it/s]


In [ ]:
pr_dict['ayasofya']

11.372075372064897

In [ ]:
df_pr = pd.DataFrame.from_dict(pr_dict, orient='index')
df_pr = df_pr.rename(columns={0: 'score'})
df_pr.head()

,score
06melihgokcek,-2.289333
rterdogan,-5.723072
olsun,4.929009
ayasofya,11.372075
tcsavunma,-5.409033


In [ ]:
df_pr.to_csv('words_pr.csv')